In [1]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from ast import literal_eval
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
import difflib
import json


In [2]:
df1=pd.read_csv('../data/archive(1)/movies_metadata.csv')

c:\Users\Vanshaj\.conda\envs\tensorflowenv_20220122\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df1 = df1.drop([19730, 29503, 35587])
df1['id'] = df1['id'].astype(int)

In [4]:
links_small = pd.read_csv('../data/archive(1)/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [5]:
new = df1['id'].isin(links_small)
df1 = df1[new]

In [6]:
df1.to_csv('df.csv')

In [7]:
df1.shape

(9099, 24)

In [8]:
C= df1['vote_average'].mean()
C

6.361006704033412

In [9]:
m = df1['vote_count'].quantile(0.9)

In [10]:
q_movies = df1.copy().loc[df1['vote_count'] >= m]

In [11]:
# pop= df1.sort_values('popularity', ascending=False)
# import matplotlib.pyplot as plt
# plt.figure(figsize=(12,4))

# plt.barh(pop['title'].head(6),pop['popularity'].head(6), align='center',
#         color='skyblue')
# plt.gca().invert_yaxis()
# plt.xlabel("Popularity")
# plt.title("Popular Movies")

In [12]:
# df1['overview'].head(5)

In [13]:
# #Import TfIdfVectorizer from scikit-learn
# from sklearn.feature_extraction.text import TfidfVectorizer

# #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
# tfidf = TfidfVectorizer(stop_words='english')

# #Replace NaN with an empty string
# df1['overview'] = df1['overview'].fillna('')
# df1['tagline'] = df1['tagline'].fillna('')

# df1['description'] = df1['overview'] + df1['tagline']
# #Construct the required TF-IDF matrix by fitting and transforming the data
# tfidf_matrix = tfidf.fit_transform(df1['description'])

# #Output the shape of tfidf_matrix
# tfidf_matrix.shape

In [14]:
# from sklearn.metrics.pairwise import linear_kernel

# # Compute the cosine similarity matrix
# cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [15]:
# indices = pd.Series(df1.index, index=df1['title']).drop_duplicates()

In [16]:
# def get_recommendations(title, cosine_sim=cosine_sim):
#     # Get the index of the movie that matches the title
#     idx = indices[title]

#     # Get the pairwsie similarity scores of all movies with that movie
#     sim_scores = list(enumerate(cosine_sim[idx]))

#     # Sort the movies based on the similarity scores
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

#     # Get the scores of the 10 most similar movies
#     sim_scores = sim_scores[1:11]

#     # Get the movie indices
#     movie_indices = [i[0] for i in sim_scores]

#     # Return the top 10 most similar movies
#     return df1['title'].iloc[movie_indices]

In [17]:
# get_recommendations('The Wolf of Wall Street')

In [18]:
df1['genres'] = df1['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [19]:
credits = pd.read_csv('../data/archive(1)/credits.csv')
keywords = pd.read_csv('../data/archive(1)/keywords.csv')

In [20]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')


In [21]:
df1['id'].value_counts()

12600     2
69234     2
265189    2
4912      2
110428    2
         ..
20770     1
4247      1
4244      1
12706     1
391698    1
Name: id, Length: 9082, dtype: int64

In [22]:
df1 = df1.merge(credits, on='id')
df1 = df1.merge(keywords, on='id')

In [23]:
df1['cast'] = df1['cast'].apply(literal_eval)
df1['crew'] = df1['crew'].apply(literal_eval)
df1['keywords'] = df1['keywords'].apply(literal_eval)
df1['cast_size'] = df1['cast'].apply(lambda x: len(x))
df1['crew_size'] = df1['crew'].apply(lambda x: len(x))

In [24]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [25]:
df1['director'] = df1['crew'].apply(get_director)

In [26]:
df1['cast'] = df1['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df1['cast'] = df1['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [27]:
df1['keywords'] = df1['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [28]:
df1['cast'] = df1['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [29]:


df1['director'] = df1['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
df1['director'] = df1['director'].apply(lambda x: [x,x, x])



In [30]:
s = df1.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

C:\Users\Vanshaj\AppData\Local\Temp/ipykernel_1128/4160031755.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = df1.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)


In [31]:
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [32]:
s = s[s > 1]

In [33]:
stemmer = SnowballStemmer('english')

In [34]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [35]:
df1['keywords'] = df1['keywords'].apply(filter_keywords)
df1['keywords'] = df1['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
df1['keywords'] = df1['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [36]:
df1['soup'] = df1['keywords'] + df1['cast'] + df1['director'] + df1['genres']
df1['soup'] = df1['soup'].apply(lambda x: ' '.join(x))

In [37]:
df1['soup']

0       jealousi toy boy friendship friend rivalri boy...
1       boardgam disappear basedonchildren'sbook newho...
2       fish bestfriend duringcreditssting waltermatth...
3       basedonnovel interracialrelationship singlemot...
4       babi midlifecrisi confid age daughter motherda...
                              ...                        
9214    friendship sidneypoitier wendycrewson jayo.san...
9215    bollywood akshaykumar ileanad'cruz eshagupta t...
9216    bollywood hrithikroshan poojahegde kabirbedi a...
9217    monster godzilla giantmonst destruct kaiju hir...
9218    music documentari paulmccartney ringostarr joh...
Name: soup, Length: 9219, dtype: object

In [38]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(df1['soup'])


In [39]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [40]:
cosine_sim1 = np.array(cosine_sim)

with open('test.npy', 'wb') as f:
    np.save(f,cosine_sim1)

In [41]:
# data = np.load('test.npy')

In [42]:
df1 = df1.reset_index()
titles = df1['title']
indices = pd.Series(df1.index, index=df1['title'])



In [43]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    # idx = indices[title]
    list_of_all_titles = df1['title'].tolist()
    closest_match = difflib.get_close_matches(title,list_of_all_titles)[0]

    idx = df1.index[df1['title']==closest_match].tolist()[0]
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df1['title'].iloc[movie_indices]

In [44]:
try:
    print(get_recommendations('Thor: Love and Thunder'))
except:
    print('No such recommendation')
    

3794                                         Zoolander
9130                                       Zoolander 2
336                                      Reality Bites
652                                      The Cable Guy
8550                   The Secret Life of Walter Mitty
4839                                            Duplex
7297    Night at the Museum: Battle of the Smithsonian
5204                                              Envy
8002                                       Tower Heist
5159                                   Starsky & Hutch
Name: title, dtype: object


https://www.kaggle.com/code/rounakbanik/movie-recommender-systems/notebook